<a href="https://colab.research.google.com/github/samuel-zahner/Intro-to-AI/blob/main/MineSweeper_inference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [114]:
import numpy as np
import numpy.random as rand

In [139]:
#create environment
#take input from user
while True:
  try:
    dimension = int(input("Enter dimension of board: "))
    break
  except ValueError:
    print("Must be an integer")

while True: 
  try:
    num_mines = int(input("Enter number of mines: "))
    if num_mines > dimension**2:
      raise ValueError
    break
  except ValueError:
    print("Must be an integer and less than dimension^2")



Enter dimension of board: 4
Enter number of mines: 2


In [116]:
#initialize board
board = np.zeros((dimension, dimension))

#initialize mines
counter = 0
mines_set = set([])
while counter < num_mines:
  x = rand.randint(0, dimension)
  y = rand.randint(0, dimension)
  if x == 0 and y == 0:
    continue
  if (x,y) in mines_set:
    continue
  else:
    mines_set.add((x,y))
    board[x][y] = 1
    counter += 1
    
#hard coding values to test cases
board[0][1] = 0
board[1][1] = 0
board[1][0] = 1
print(board)

[[0. 0. 0. 0.]
 [1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [1. 0. 0. 0.]]


In [117]:
#clue that our agent will receive
#returns the total sum of the neighbors of current_position
def clue(current_position, board,  dimension):
  x = current_position[0]
  y = current_position[1]
  sum = 0
  
  if x+1 < dimension:
    sum += board[x+1][y]
    if y+1 < dimension:
      sum += board[x+1][y+1]
    if y-1 >= 0:
      sum += board[x+1][y-1]

  if x-1 >= 0:
    sum += board[x-1][y]
    if y+1 < dimension:
      sum += board[x-1][y+1]
    if y-1 >= 0:
      sum += board[x-1][y-1]

  if y+1 < dimension:
    sum += board[x][y+1]
  
  if y-1 >= 0:
    sum += board[x][y-1]

  return sum
  

In [143]:
start = (0,0)
board_perception = np.zeros((dimension, dimension)) #-1: unknown, 0: clear, 1: mine
clue_perception = np.zeros((dimension, dimension))
for i in range(dimension):
  for j in range(dimension):
    board_perception[i][j] = -1
    clue_perception[i][j] = -1

knowledge_base = [(0, 0, clue(start, board, dimension)), (0, 1, 1), (1, 1, 1)] #(0,0): location, 0: value(0 or 1)
board_perception[start[0]][start[1]] = 0 #(known value)
clue_perception[start[0]][start[1]] = clue(start, board, dimension)
board_perception[0,1] = 0
board_perception[1,1] = 0

get_neighbors = []
for i in range(-1,2):
  for j in range(-1,2):
    if i==0 and j==0:
      continue
    get_neighbors.append((i,j))



while(-1 in board_perception):
  #loop through neighbors of current_position asserting if they are 0(clear) and 1(mine)
  #compare each assertion to current knowledge_base, use proof by contradiction to determine if true
  #update knowledge base
  #move current_position, check if mine, update board_perception, compute new clue
  #continue loop


  break
#can play around with query of (1,0)
query = (1, 0, 1)
#False = there was a contradiction found
#True = no contradiction
make_inference(knowledge_base, board_perception, query)

Length variable list: 4
[(0, 1), (1, 1), (1, 0), (0, 0)]
var: 0
var_binary: 0b0000
Inner for loop i: 2
Inner for loop i: 3
Inner for loop i: 4
Inner for loop i: 5
end variable dictionary: {(0, 1): 0, (1, 1): 0, (1, 0): 0, (0, 0): 0}
var: 1
var_binary: 0b0001
Inner for loop i: 2
Inner for loop i: 3
Inner for loop i: 4
Inner for loop i: 5
end variable dictionary: {(0, 1): 0, (1, 1): 0, (1, 0): 0, (0, 0): 1}
var: 2
var_binary: 0b0010
Inner for loop i: 2
Inner for loop i: 3
Inner for loop i: 4
Inner for loop i: 5
end variable dictionary: {(0, 1): 0, (1, 1): 0, (1, 0): 0, (0, 0): 0}
var: 3
var_binary: 0b0011
Inner for loop i: 2
Inner for loop i: 3
Inner for loop i: 4
Inner for loop i: 5
end variable dictionary: {(0, 1): 0, (1, 1): 0, (1, 0): 0, (0, 0): 1}
var: 4
var_binary: 0b0100
Inner for loop i: 2
Inner for loop i: 3
Inner for loop i: 4
Inner for loop i: 5
end variable dictionary: {(0, 1): 0, (1, 1): 1, (1, 0): 0, (0, 0): 0}
var: 5
var_binary: 0b0101
Inner for loop i: 2
Inner for loop i:

False

In [122]:
#evaluate an assertion in knowledge_base 
#variables contains new assertion
def evaluate_assertion(assertion, variables, dimension):
  x,y,v = assertion #(x,y) -> location, v -> value of cell( 0 or 1)

  sum = 0
  for i in get_neighbors:
    neighbor_x = x + i[0]
    neighbor_y = y + i[1]
    value = board_perception[neighbor_x][neighbor_y]

    if neighbor_x >= dimension or neighbor_x < 0:
      continue
    elif neighbor_y >= dimension or neighbor_y < 0:
      continue
    #add values of known cells
    elif value != -1:
      sum += value
    #if neighbor is in variables dictionary, add its value
    elif (neighbor_x, neighbor_y) in variables:
      sum += variables[(neighbor_x, neighbor_y)]
    #cannot compute sum of unkown vales so return truw
    else:
      return True
  return sum == v



In [137]:
#will query all neighbors of current position and call evaluate_assertion for each assertion in knowledge_base
def make_inference(knowledge_base, board_perception, query):
  qx, qy, qv = query

  variable_list = []

  #loop through assertions in knowledge base
  for assertion in knowledge_base:
    x, y, v = assertion

    #loop through all possible neighbors
    for i in get_neighbors:
      neighbor_x = x + i[0]
      neighbor_y = y + i[1]
      value = board_perception[neighbor_x][neighbor_y]

      if neighbor_x >= dimension or neighbor_x < 0:
        continue
      elif neighbor_y >= dimension or neighbor_y < 0:
        continue
      #if cell not hidden -> add to variables
      elif (neighbor_x, neighbor_y) in variable_list:
        continue
      elif value != -1:
        variable_list.append((neighbor_x, neighbor_y))


    #add query to variables
    if (qx, qy) not in variable_list:
      variable_list.append((qx, qy))

  #constraint satisfaction problem -> loop through all possible assignments and find one solution
  #since all possible solutions are 0 or 1, use binary
  satisfiable = False
  print("Length variable list: " + str(len(variable_list)))
  print(variable_list)

  #loops through 2^len(variable_list) because for n variables there are 2^n combinations of values
  for var in range(2**(len(variable_list))):
    print("var: " + str(var))
    var_binary = format(var, '#0%db' % (len(variable_list) + 2))
    print("var_binary: " + str(var_binary))
    var_dict = {}

    #start looping at index = 2 because of '0b' prefix on binary 
    for i in range(2, len(var_binary)):
      print("Inner for loop i: " + str(i))
      var_dict[variable_list[i-2]] = int(var_binary[i])
      #q -> not q
      if variable_list[i-2] == (qx,qy):
        var_dict[variable_list[i-2]] = 1-qv

    print("end variable dictionary: " + str(var_dict)) 

    flag = True
    for assertion in knowledge_base:
      #check each assertion in knowledge base with our contradicted query
      truth_val = evaluate_assertion(assertion, var_dict, dimension)

      if not truth_val:
        flag = False
        break
    
    if flag:
      satisfiable = True
      break
  #return satisfiability of knowledge base
  return satisfiable
      


  